In [1]:
import csv
import json
import logging
import math

import tqdm
from shapely.geometry import Point, LineString

In [2]:
with open("tomskaia-oblast.geojson", "rt") as f:
    dtp = json.load(f)

In [3]:
len(dtp["features"])

4718

In [4]:
dtp["features"][0].keys()

dict_keys(['type', 'geometry', 'properties'])

In [5]:
# # cut-off DTP related to Tomsk city
# tomsk_boundaries = {"min": {"lat": 56.4, "lon": 84.8}, "max": {"lat": 56.56, "lon": 85.1}}

# def check_boundary(item):
#     lon, lat = item["geometry"]["coordinates"]
#     if not lon or not lat:
#         logging.warning(f'Unknown coordinates! Region: {item["properties"]["region"]}.')
#         return False
#     return (tomsk_boundaries["min"]["lat"] < lat < tomsk_boundaries["max"]["lat"]
#            and tomsk_boundaries["min"]["lon"] < lon < tomsk_boundaries["max"]["lon"])

def check_region(item):
    return item["properties"]["region"] == "Томск"

dtp_tomsk = list(filter(check_region, dtp["features"]))
print(f'всего ДТП в регионе "Томск": {len(dtp_tomsk)}')

всего ДТП в регионе "Томск": 2318


In [6]:
roads = []
with open("00_roads.csv", "rt") as f:
    reader = csv.reader(f)
    header = next(reader)
    print("Header columns:", header)
    geometry_index = header.index("geometry_json")
    for row in reader:
        # parse geojson object from string
        row[geometry_index] = json.loads(row[geometry_index])
        roads.append(dict(zip(header,row)))

Header columns: ['type', 'worktypeid', 'status', 'geometry_json', 'victoryroad', 'title', 'id_fragment']


In [7]:
len(roads)

209

In [8]:
list(filter(lambda x: x["geometry_json"]["type"]=="Point", roads))[0],\
list(filter(lambda x: x["geometry_json"]["type"]=="LineString", roads))[0]

({'type': 'Feature',
  'worktypeid': '62684cfd-a975-427d-a0de-2809e4448468',
  'status': '2',
  'geometry_json': {'type': 'Point',
   'coordinates': [84.87588349547373, 56.60275976399258]},
  'victoryroad': 'False',
  'title': 'автодорога "Северные автомогистраль ПК 0 по  ПК 21+90',
  'id_fragment': '4d038213-f907-4c69-94e3-c55399866954'},
 {'type': 'Feature',
  'worktypeid': '88385f78-2a6e-4c60-94e9-4b63f24c3b85',
  'status': '2',
  'geometry_json': {'type': 'LineString',
   'coordinates': [[81.00193977355958, 58.88337003335058],
    [81.00138187408447, 58.8872980422736],
    [80.99940776824953, 58.891558428860755],
    [80.99863529205324, 58.89397685223988],
    [81.00125312805177, 58.89679443490843],
    [80.99876403808595, 58.901652561663],
    [80.9997081756592, 58.906088610587624],
    [80.99799156188966, 58.90886085200609],
    [80.98653316497804, 58.91761967304173],
    [80.97357273101808, 58.928038668306876],
    [80.9716844558716, 58.929124723036125],
    [80.95846652984619, 

In [9]:
def get_distance(coord, geo_obj) -> float:
    """returns distance (in km) from Point `coord` to `geo_obj`
    geo_obj: geojson object with type Point or LineString"""
    
    coord_lon, coord_lat = coord
    
    grad_lat = 111.111  # km/gradus
    grad_lon = (40000/360)*math.cos(coord_lat/180*math.pi)
    
    correction = grad_lon/grad_lat
    
    flat_coord = Point([coord_lon*correction, coord_lat])
    
    if geo_obj["type"] == "Point":
        flat_obj = Point([geo_obj["coordinates"][0]*correction, geo_obj["coordinates"][1]])
    elif geo_obj["type"] == "LineString" and len(geo_obj["coordinates"]) == 1:
        # line with only one point, will handle as Point
        flat_obj = Point([geo_obj["coordinates"][0][0]*correction, geo_obj["coordinates"][0][1]])
    elif geo_obj["type"] == "LineString":
        flat_obj = LineString([[p[0]*correction, p[1]] for p in geo_obj["coordinates"]])
    else:
        raise ValueError(f"Unsupported type of geo_obj: {geo_obj}!")
    return flat_coord.distance(flat_obj) * grad_lat

# tests:
# get_distance([79,59], {"type": "LineString", "coordinates": [[80,59],[81,59]]}) -> 57.22645276778391
# get_distance([80,58], {"type": "LineString", "coordinates": [[80,59],[81,59]]}) -> 111.111


In [10]:
allowed_distance = 0.050  # DTP within this range (in km) will be associated with repaired road
collected_dtp = []

for dtp in tqdm.tqdm(dtp_tomsk):
    dtp_id = dtp["properties"]["id"]
    dtp_lat = dtp["properties"]["point"]["lat"]
    dtp_lon = dtp["properties"]["point"]["long"]
    if not dtp_lat or not dtp_lon:
        logging.warning(f"No coords for DTP id:{dtp_id}!")
        continue
        
    # choose only one road repair which is closest to DTP coords
#     collected_remonts = []
    found_fragment = None
    found_min_dist = 2 * allowed_distance  # set default to a number, larger than allowed threshold
    for remont in roads:
        dtp_distance = get_distance([dtp_lon, dtp_lat], remont["geometry_json"])
        if  dtp_distance < allowed_distance and dtp_distance < found_min_dist:
            found_min_dist = dtp_distance
#             collected_remonts.append(remont["id_fragment"])
            found_fragment = remont["id_fragment"]
#     collected_remonts = list(set(collected_remonts))
#     collected_dtp.append({"dtp_id": dtp_id, "fragments": collected_remonts})
    collected_dtp.append({"dtp_id": dtp_id, "fragment": found_fragment})

100%|██████████| 2318/2318 [00:09<00:00, 238.47it/s]


In [11]:
with open("01_dtp_to_fragments.json", "wt+") as f:
    json.dump(collected_dtp, f, indent=2)

In [12]:
list(filter(lambda x: x["properties"]["id"]==2281407, dtp_tomsk))

[{'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [84.965365, 56.537441]},
  'properties': {'id': 2281407,
   'tags': ['Дорожно-транспортные происшествия'],
   'light': 'Светлое время суток',
   'point': {'lat': 56.537441, 'long': 84.965365},
   'nearby': ['Остановка общественного транспорта',
    'Одиночный торговый объект, являющийся местом притяжения транспорта и (или) пешеходов'],
   'region': 'Томск',
   'address': 'г Томск, ул Кутузова, 1а',
   'weather': ['Ясно'],
   'category': 'Наезд на стоящее ТС',
   'datetime': '2018-09-07 17:42:00',
   'severity': 'Тяжёлый',
   'vehicles': [{'year': 1989,
     'brand': 'ВАЗ',
     'color': 'Серый',
     'model': ' Жигули  ВАЗ-2106 модификации',
     'category': 'С-класс (малый средний, компактный) до 4,3 м',
     'participants': [{'role': 'Водитель',
       'gender': 'Мужской',
       'violations': ['Управление ТС лицом, лишенным права управления',
        'Управление ТС в состоянии наркотического опьянения',
        'Дру